In [18]:
import torch
import torchvision
from torchvision import models,datasets,transforms
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [20]:
#sos remember to change the directory for the tensors
dir = '../data/dataset_erasing_02'

params = { 'batch_size':16,
           'shuffle':True,
           'num_workers':4 }


transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomResizedCrop(256), #Augmented
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#the training dataset will be from the datase_balanced folder 
train_dataset = datasets.ImageFolder(os.path.join(dir, 'train'),transform = transform )

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


#change the dir for validationa (and test)
dir = '../data/dataset_split'
val_dataset = datasets.ImageFolder(os.path.join(dir, 'val'),transform = transform )


#data loaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, **params)
val_dataloader = torch.utils.data.DataLoader(val_dataset, **params)

class_names = train_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
print('Train dataset = {}\n'.format(len(train_dataset)),'Val dataset = {}'.format(len(val_dataset)))
print('Classes = {}'.format(class_names))

Train dataset = 8832
 Val dataset = 719
Classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [22]:
import torch
import pandas as pd
import csv

def train(model, loss_fn, optimizer, num_epochs=25, save_path='training_realwaste_erasing03_data_augmentation_original_validation_set_results.csv'):
    
    best_acc = 0
    results = []  # Store epoch-wise results as lists of arrays

    for epoch in range(num_epochs):
        
        print('Epoch {}'.format(epoch+1))
        
        # Train dataset
        model.train()
        train_loss = 0.0
        train_correct = 0
        size = len(train_dataset)
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            train_correct += torch.sum(preds == labels.data)
            
        train_loss = train_loss / size
        train_acc = train_correct.double() / size
            
        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc))
        
        # Validation dataset
        model.eval()
        val_loss = 0.0
        val_correct = 0
        size = len(val_dataset)
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_fn(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += torch.sum(preds == labels.data) 

        val_loss = val_loss / size
        val_acc = val_correct.double() / size    

        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))    
            
        if val_acc > best_acc:
            best_acc = val_acc

        # Store the results for this epoch as a list of arrays
        results.append([epoch + 1, [train_loss], [train_acc.item()], [val_loss], [val_acc.item()]])
    
    # Save results to CSV
    df = pd.DataFrame(results, columns=['Epoch', 'Train Loss', 'Train Accuracy', 'Validation Loss', 'Validation Accuracy'])
    df.to_csv(save_path, index=False)
    
    print('Best Validation Accuracy: {:.4f}'.format(best_acc))
    print(f'Training results saved to {save_path}')
    
    return model

In [23]:
net = torchvision.models.densenet121(pretrained=True)
for param in net.parameters():
    param.requires_grad = False # freeze all the weights

ft = net.classifier.in_features # final layer of the densenet
net.classifier = nn.Linear(ft, 6) # new layer according to our dataset with weights unfrozen

net = net.to(device)

loss = nn.CrossEntropyLoss()

# only final layer optimized
optimizer = optim.SGD(net.classifier.parameters(), lr=0.0001, momentum=0.9) 

In [24]:
net = train(net,loss,optimizer,num_epochs=100)



Epoch 1
Training Loss: 1.6927 Acc: 0.2981
Validation Loss: 1.4403 Acc: 0.4715
Epoch 2
Training Loss: 1.5366 Acc: 0.4101
Validation Loss: 1.2576 Acc: 0.5939
Epoch 3
Training Loss: 1.4493 Acc: 0.4600
Validation Loss: 1.1712 Acc: 0.6273
Epoch 4
Training Loss: 1.3945 Acc: 0.4922
Validation Loss: 1.1087 Acc: 0.6467
Epoch 5
Training Loss: 1.3517 Acc: 0.5120
Validation Loss: 1.0309 Acc: 0.6634
Epoch 6
Training Loss: 1.3262 Acc: 0.5190
Validation Loss: 1.0248 Acc: 0.6745
Epoch 7
Training Loss: 1.3195 Acc: 0.5082
Validation Loss: 0.9738 Acc: 0.6940
Epoch 8
Training Loss: 1.2819 Acc: 0.5296
Validation Loss: 0.9998 Acc: 0.6787
Epoch 9
Training Loss: 1.2756 Acc: 0.5290
Validation Loss: 0.9241 Acc: 0.7121
Epoch 10
Training Loss: 1.2604 Acc: 0.5405
Validation Loss: 0.9363 Acc: 0.6940
Epoch 11
Training Loss: 1.2409 Acc: 0.5437
Validation Loss: 0.9251 Acc: 0.6857
Epoch 12
Training Loss: 1.2377 Acc: 0.5376
Validation Loss: 0.9090 Acc: 0.6787
Epoch 13
Training Loss: 1.2324 Acc: 0.5459
Validation Loss: 0

Saving the parameters of the desnet model after training!

In [25]:
import torch

torch.save(net.state_dict(), "trained_realwaste_erasing_version_03_val_densenet.pth")  # Save model weights
print("Model saved as trained_densenet.pth")


Model saved as trained_densenet.pth
